In [2]:
from os import listdir
from os.path import isfile, join
# from Utils import Load, Write
import pandas as pd
import numpy as np
from scipy import stats
# import math
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
# from matplotlib import cm
# import plotly.plotly as py
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC

In [5]:
def ReadAllFiles():
    onlyfiles = [f for f in listdir("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles") if isfile(join("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles", f))]
    onlyfiles = [datafile for datafile in onlyfiles if datafile.startswith ('HIP') and datafile.endswith('.csv')]
    countfiles=0
    for datafile in onlyfiles[12:]:
        print datafile
        countfiles=countfiles+1
        sample_name=re.sub('.csv', '', datafile)
        get_sample_data(sample_name)
   
        

In [4]:
onlyfiles = [f for f in listdir("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles") if isfile(join("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles", f))]
onlyfiles = [datafile for datafile in onlyfiles if datafile.startswith ('HIP') and datafile.endswith('.csv')]
countfiles=0

In [ ]:
print onlyfiles.index('HIP13554.csv')

In [6]:
def get_sample_data(sample_name, generate_dfs): 
## this function generates dfs (general, only productive and only non productive) for each sample, and save
## as pickles. alternatively, it loads these dfs from pickles. 
    
    print 'getting sample data...'
    
    if generate_dfs:
        print 'generating'
        sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/%s.tsv" %sample_name)  
        sample_df_prod = sample_df[sample_df['sequenceStatus'] == 'In']
        sample_df_non_prod = sample_df[sample_df['sequenceStatus'] != 'In']
        
        pickle.dump(sample_df, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_%s' %sample_name, "wb"))
        pickle.dump(sample_df_prod, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_prod_%s' %sample_name, "wb"))
        pickle.dump(sample_df_non_prod, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_non_prod_%s' %sample_name, "wb"))
    else:
        print 'loading'
        sample_df= pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_%s' %sample_name,"rb" ))
        sample_df_prod= pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_prod_%s' %sample_name,"rb" ))
        sample_df_non_prod= pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/sample_df_non_prod_%s' %sample_name,"rb" ))
    
    print 'finished getting sample data'
    return sample_df, sample_df_prod, sample_df_non_prod


In [ ]:
def unique_seq_n(df): 
    n_seqs=len(list(df['nucleotide']))
    return n_seqs

def unique_aa_n(df):
    list_aa=list(df['aminoAcid'])
    list_aa_new=[i for i in list_aa if isinstance(i, str)]
    n_seq_aa=len(set(list_aa_new))
    return n_seq_aa


def max_nt_per_aa(df):
    nt_per_aa = df.groupby(['aminoAcid'])[['nucleotide']].count()
    max_nt_per_aa=max(nt_per_aa['nucleotide'])
    return max_nt_per_aa

def mean_nt_per_aa(df):
    nt_per_aa = df.groupby(['aminoAcid'])[['nucleotide']].count()
    mean_nt_per_aa=round(np.mean(nt_per_aa['nucleotide']),3)
    return mean_nt_per_aa

def generate_col_dict(df_n, function_list):
    function_names=[f.__name__ for f in function_list]
    ind=0
    col_dict={}
    for i in range(df_n):
        for j in range(len(function_names)):
            col_dict[ind]='func-%s_df#%s' %(j, i)
            ind+=1
    return col_dict
    



def calculate_general(results_gen_df,function_list, sample_name, sample_df, sample_df_prod, sample_df_non_prod):
    print 'calculating general features...'
    ind=0
    df_list=[sample_df_prod, sample_df_non_prod]
    for i, df in enumerate(df_list):
        for j, func in enumerate(function_list):
            result=func(df)
            results_gen_df.loc[sample_name,[ind]]=result
            ind+=1
    print 'finished calculating general features...'
    return results_gen_df

def calculate_clonality(results_clon_df,top_n_list, sample_name, sample_df, sample_df_prod, sample_df_non_prod):
    print 'calculating clonality features...'
    ind=0
    df_list=[sample_df_prod, sample_df_non_prod]
    for i, df in enumerate(df_list):
        for j, n in enumerate(top_n_list):
            result=top_clonal(gen_clonality_list(df), n)
            results_clon_df.loc[sample_name,[ind]]=result
            ind+=1
    print 'finished calculating clonality features...'
    return results_clon_df

top1=top_clonal(gen_clonality_list(sample_df_prod),1)

def get_sample_with_dfs():
    df_file_names = [f for f in listdir("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles") if isfile(join("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles", f))]
    df_file_names = [df_file for df_file in df_file_names if df_file.startswith ('sample_df_HIP')]
    samples_with_df=[s.replace("sample_df_", "") for s in df_file_names]
    return df_file_names,samples_with_df

def general_plot(function_list, results_gen_df,yscale):
    print 'generating general features plot...'
    gen_fig=plt.figure(figsize=(10,8))
    plt.suptitle('General population features', fontsize=16)
    function_names=[f.__name__ for f in function_list]
    n_plots=len(function_list)
    for p in range(n_plots):
        title=function_names[p]
        prod_col=p
        non_prod_col=p+4
        ax= plt.subplot2grid((n_plots,1), (p,0))
        print('ax is %s' %ax)
        plot=plot_population_view(ax,results_gen_df, prod_col,non_prod_col, title,yscale) 
    plt.subplots_adjust(left=0.12,bottom=0.04, right=0.97, top=0.90, wspace=0.28,hspace=0.60)      
    plt.show()
    #with PdfPages('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Images/Population_View_general_features') as pdf:
    #    pdf.savefig(gen_fig)
    #pdf.close
    

    

def plot_population_view(ax,results_gen_df, prod_col,non_prod_col,title,yscale):
    n_bins=50
    print('prod_col is %s' %str(prod_col))
    print('non_prod_col is %s' %str(non_prod_col))
    prod_l=list(results_gen_df[prod_col])
    while 0 in prod_l: prod_l.remove(0)
    print prod_l[:10]
    non_prod_l=list(results_gen_df[non_prod_col])
    while 0 in non_prod_l: non_prod_l.remove(0)
    print non_prod_l[:10]
    plot=ax.hist((prod_l,non_prod_l), color=('black', 'red'), label=('Productive','Non-Productive'),bins=n_bins)
    ax.set_title(title, fontsize=14)
    ax.set_ylabel('Frequency')
    ax.set_yscale(yscale)
    if prod_col==0:
        ax.legend(loc='upper right', fontsize=9)
    return plot
    
    
    


In [14]:
def unique_seq_n(df): 
    n_seqs=len(list(df['nucleotide']))
    return n_seqs

def unique_aa_n(df):
    list_aa=list(df['aminoAcid'])
    list_aa_new=[i for i in list_aa if isinstance(i, str)]
    n_seq_aa=len(set(list_aa_new))
    return n_seq_aa


def max_nt_per_aa(df):
    nt_per_aa = df.groupby(['aminoAcid'])[['nucleotide']].count()
    max_nt_per_aa=max(nt_per_aa['nucleotide'])
    return max_nt_per_aa

def mean_nt_per_aa(df):
    nt_per_aa = df.groupby(['aminoAcid'])[['nucleotide']].count()
    mean_nt_per_aa=round(np.mean(nt_per_aa['nucleotide']),3)
    return mean_nt_per_aa



In [ ]:
def generate_col_dict(df_n, function_list):
    function_names=[f.__name__ for f in function_list]
    ind=0
    col_dict={}
    for i in range(df_n):
        for j in range(len(function_names)):
            col_dict[ind]='func-%s_df#%s' %(j, i)
            ind+=1
    return col_dict
    


In [ ]:
def calculate_general(results_gen_df,function_list, sample_name, sample_df, sample_df_prod, sample_df_non_prod):
    print 'calculating general features...'
    ind=0
    df_list=[sample_df_prod, sample_df_non_prod]
    for i, df in enumerate(df_list):
        for j, func in enumerate(function_list):
            result=func(df)
            results_gen_df.loc[sample_name,[ind]]=result
            ind+=1
    print 'finished calculating general features...'
    return results_gen_df

def general_plot(function_list, results_gen_df,yscale):
    print 'generating general features plot...'
    gen_fig=plt.figure(figsize=(10,8))
    plt.suptitle('General population features', fontsize=16)
    function_names=[f.__name__ for f in function_list]
    n_plots=len(function_list)
    for p in range(n_plots):
        title=function_names[p]
        prod_col=p
        non_prod_col=p+4
        ax= plt.subplot2grid((n_plots,1), (p,0))
        print('ax is %s' %ax)
        plot=plot_population_view(ax,results_gen_df, prod_col,non_prod_col, title,yscale) 
    plt.subplots_adjust(left=0.12,bottom=0.04, right=0.97, top=0.90, wspace=0.28,hspace=0.60)      
    plt.show()
    #with PdfPages('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Images/Population_View_general_features') as pdf:
    #    pdf.savefig(gen_fig)
    #pdf.close
    
    

def plot_population_view(ax,results_gen_df, prod_col,non_prod_col,title,yscale):
    n_bins=50
    print('prod_col is %s' %str(prod_col))
    print('non_prod_col is %s' %str(non_prod_col))
    prod_l=list(results_gen_df[prod_col])
    while 0 in prod_l: prod_l.remove(0)
    print prod_l[:10]
    non_prod_l=list(results_gen_df[non_prod_col])
    while 0 in non_prod_l: non_prod_l.remove(0)
    print non_prod_l[:10]
    plot=ax.hist((prod_l,non_prod_l), color=('black', 'red'), label=('Productive','Non-Productive'),bins=n_bins)
    ax.set_title(title, fontsize=14)
    ax.set_ylabel('Frequency')
    ax.set_yscale(yscale)
    if prod_col==0:
        ax.legend(loc='upper right', fontsize=9)
    return plot


In [7]:
def get_sample_with_dfs():
    df_file_names = [f for f in listdir("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles") if isfile(join("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles", f))]
    df_file_names = [df_file for df_file in df_file_names if df_file.startswith ('sample_df_HIP')]
    samples_with_df=[s.replace("sample_df_", "") for s in df_file_names]
    return df_file_names,samples_with_df


In [ ]:
def calculate_clonality(results_clon_df,top_n_list, sample_name, sample_df, sample_df_prod, sample_df_non_prod):
    print 'calculating clonality features...'
    ind=0
    df_list=[sample_df_prod, sample_df_non_prod]
    for i, df in enumerate(df_list):
        for j, n in enumerate(top_n_list):
            result=top_clonal(gen_clonality_list(df), n)
            results_clon_df.loc[sample_name,[ind]]=result
            ind+=1
    print 'finished calculating clonality features...'
    return results_clon_df

top1=top_clonal(gen_clonality_list(sample_df_prod),1)




def maximal_clonality(df):
    max_freq=df['frequencyCount (%)'].max()
    total_freq=df['frequencyCount (%)'].sum()
    max_clonality_normed=max_freq/total_freq*100
    return max_clonality_normed, total_freq

def gen_clonality_list(df):
    clonality_list=list(df['frequencyCount (%)'])
    clonality_total=sum(clonality_list)
    normed_clonality_list=[(i/clonality_total)*100 for i in clonality_list]
    return normed_clonality_list

def top_clonal(normed_clonality_list,n):
    sorted_clon_list=sorted(normed_clonality_list, reverse=True)
    print sorted_clon_list[:10]
    fraction=0
    for i in range(n):
        print i
        fraction+=sorted_clon_list[i]
        print fraction
    for j in range(n,len(sorted_clon_list)):
        print j
        while sorted_clon_list[j]==sorted_clon_list[j-1]:
            fraction+=sorted_clon_list[j] 
            print fraction
    return fraction
    
    
    
    


In [8]:
df_file_names,samples_with_df=get_sample_with_dfs()
#get list of samples that have df

In [9]:
'HIP13554'in samples_with_df
#check whether the desired sample has df

True

In [13]:
sample_df, sample_df_prod, sample_df_non_prod=get_sample_data('HIP13554', False)

getting sample data...
loading
finished getting sample data


In [14]:
sample_df_prod.head()

,nucleotide,aminoAcid,count (reads),frequencyCount (%),cdr3Length,vMaxResolved,vFamilyName,vGeneName,vGeneAllele,vFamilyTies,...,estimatedNumberGenomes,sequenceStatus,cloneResolved,vOrphon,dOrphon,jOrphon,vFunction,dFunction,jFunction,fractionNucleated
0,AGGCTGCTGTCGGCTGCTCCCTCCCAGACATCTGTGTACTTCTGTG...,CASSLLARDTQYF,670863,8.992874,39,TCRBV06-05*01,TCRBV06,TCRBV06-05,1.0,NaN,...,21880,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,0.230093
1,TCGGCTGCTCCCTCCCAAACATCTGTGTACTTCTGTGCCAGCAGTT...,CASSYPSGTGWIEQFF,132753,1.779545,48,TCRBV06,TCRBV06,unresolved,NaN,NaN,...,4315,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,0.045532
3,ATCCAGCGCACAGAGCAGGGGGACTCGGCCATGTATCTCTGTGCCA...,CASSPPTGSYEQYF,63804,0.855288,42,TCRBV07-09,TCRBV07,TCRBV07-09,NaN,NaN,...,2070,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,0.021884
4,AGCACCTTGGAGCTGGGGGACTCGGCCCTTTATCTTTGCGCCAGCA...,CASSDDRGQGETQYF,63007,0.844605,45,TCRBV05-01*01,TCRBV05,TCRBV05-01,1.0,NaN,...,2050,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,0.021610
5,GCAAAGCTTGAGGACTCGGCCGTGTATCTCTGTGCCAGCAGCTTAG...,CASSLDREPQSNQPQHF,62751,0.841173,51,TCRBV11-02*02,TCRBV11,TCRBV11-02,2.0,NaN,...,2044,In,VDJ,NaN,NaN,NaN,NaN,NaN,NaN,0.021522


In [16]:
sample_df_prod.columns[0]

'nucleotide'

In [ ]:
top1=top_clonal(gen_clonality_list(sample_df_prod),1)
top10=top_clonal(gen_clonality_list(sample_df_prod),10)
top50=top_clonal(gen_clonality_list(sample_df_prod),50)
top99=top_clonal(gen_clonality_list(sample_df_prod),99)
print top1
print top10
print top50
print top99


In [ ]:
top_n_list=[1,10,50,99]
calculate_clonality(results_clon_df,top_n_list, sample_name, sample_df, sample_df_prod, sample_df_non_prod)

In [ ]:

#########################
#    main function      #
#########################

##generate list of sample file names:
onlyfiles = [f for f in listdir("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles") if isfile(join("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/CSVfiles", f))]
onlyfiles = [datafile for datafile in onlyfiles if datafile.startswith ('HIP') and datafile.endswith('.csv')]
countfiles=0
sample_names=[re.sub('.csv', '', datafile) for datafile in onlyfiles] ## generate list of 
                                                                      ## sample names out 
                                                                      ##of the sample file names

generate_dfs=False ##False=load them from pickles, True=generate them. 

df_n=2 ## define number of dfs (prod, non-prod) *** check for update!***
function_list=[unique_seq_n, unique_aa_n,max_nt_per_aa,mean_nt_per_aa] ##*** check for update!***
col_dict=generate_col_dict(df_n, function_list)

##define the dataframe for collecting the general feature data over all samples:
results_gen_df=pd.DataFrame({'Sample': sample_names})
results_gen_df=results_gen_df.set_index('Sample')
## generate columns for each expected feature and fill them with 0's:
c=len(function_list)*df_n
for ind in range(c):
    results_gen_df[ind]=0

df_file_names,samples_with_df=get_sample_with_dfs() ## check which samples have df
no_df_samples=[]
for d in range(len(samples_with_df)): ##***change here for more sample!***
    countfiles=countfiles+1
    print countfiles
    sample_name=samples_with_df[d]
    print sample_name
## extract prod and non-prod dfs and calculate general parameters only for samples that
## have dfs:
    if sample_name not in samples_with_df:
        print('sample %s has no df' %sample_name)
        no_df_samples.append(sample_name)
    else:
        print('sample %s has df' %sample_name)
        sample_df, sample_df_prod, sample_df_non_prod=get_sample_data(sample_name, generate_dfs)
        results_gen_df=calculate_general(results_gen_df,function_list, sample_name, sample_df, sample_df_prod, sample_df_non_prod)
print results_gen_df
pickle.dump(results_gen_df, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/results_gen_df', "wb"))
pickle.dump(no_df_samples, open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/no_df_samples', "wb"))

general_plot()

   




    
    

In [ ]:
sample_names=[re.sub('.csv', '', datafile) for datafile in onlyfiles]
results_gen_df=pd.DataFrame({'Sample': sample_names})
results_gen_df=results_gen_df.set_index('Sample')
print results_gen_df

In [ ]:
results_gen_df['trial']=0
for d in onlyfiles[:10]:
    results_gen_df.loc[d,['trial']]=5
    
    

In [ ]:
df_file_names = [f for f in listdir("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles") if isfile(join("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles", f))]
df_file_names = [df_file for df_file in df_file_names if df_file.startswith ('sample_df_HIP')]
samples_with_df=[s.replace("sample_df_", "") for s in df_file_names]
print len(df_file_names)
print len(samples_with_df)

In [ ]:
exist=0
total=0
for sample in sample_names:
    if sample not in samples_with_df:
        pass
    else:
        exist+=1
print exist
print total

In [ ]:
function_list

In [ ]:
results_gen_df=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/results_gen_df', "rb"))

In [ ]:
results_gen_df

In [ ]:
general_plot(function_list, results_gen_df)

In [ ]:
results_gen_df=results_gen_df.replace(0, np.nan)

In [ ]:
results_gen_df=results_gen_df.replace(np.nan,0)

In [1]:
print results_gen_df

NameError: name 'results_gen_df' is not defined

In [ ]:
a=list(results_gen_df['0'])
b=list(results_gen_df['4'])
while 0 in a:
    a.remove(0)
while 0 in b:
    b.remove(0)
print a
print b

In [ ]:
plt.hist((a,b))
plt.show()

In [ ]:
plt.hist((a,b),label=('Prod', 'Non-Prod'))
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
plt.hist((a,b),label=('Prod', 'Non-Prod'))
plt.legend()
plt.show()

In [ ]:
plt.hist((a,b),label=('Prod', 'Non-Prod'), bins=100)
plt.legend()
plt.show()

In [ ]:
plt.hist((a,b),label=('Prod', 'Non-Prod'), bins=40)
plt.legend()
plt.show()

In [ ]:
general_plot(function_list, results_gen_df)

In [ ]:
general_plot(function_list, results_gen_df)

In [ ]:
general_plot(function_list, results_gen_df, 'log')

In [ ]:
general_plot(function_list, results_gen_df, 'log')

In [ ]:
print sample_name

In [ ]:
print sample_df_prod[:5]

In [ ]:
print maximal_clonality(sample_df_prod)

In [ ]:
print maximal_clonality(sample_df_non_prod)

In [ ]:
general_plot(function_list, results_gen_df, 'log')

In [ ]:
maximal_clonality(sample_df_non_prod)

In [ ]:
l=[1,1,1,2,4,2,5]

In [ ]:
sortedl=l.sort(reverse=True)

In [34]:
def generate_res_DF(sample_names, function_list, df_n):
    result_df=pd.DataFrame({'Sample': sample_names})
    result_df=result_df.set_index('Sample')
    function_names=[f.__name__ for f in function_list]
    print result_df.head()
    c=len(function_list)*df_n
    for function in function_names:
        for i in range(df_n):
            col_title=function+'_df_%s' %str(i)
            result_df[col_title]=0
    print result_df.head()
    return result_df

In [12]:
sample_names=samples_with_df[0:20]

In [15]:
function_list=[unique_seq_n, unique_aa_n,max_nt_per_aa,mean_nt_per_aa]

In [16]:
df_n=2

In [35]:
trial_df=generate_res_DF(sample_names, function_list, df_n)

Empty DataFrame
Columns: []
Index: [HIP11513, HIP08200, HIP00769, HIP09122, HIP12091]
          unique_seq_n_df_0  unique_seq_n_df_1  unique_aa_n_df_0  \
Sample                                                             
HIP11513                  0                  0                 0   
HIP08200                  0                  0                 0   
HIP00769                  0                  0                 0   
HIP09122                  0                  0                 0   
HIP12091                  0                  0                 0   

          unique_aa_n_df_1  max_nt_per_aa_df_0  max_nt_per_aa_df_1  \
Sample                                                               
HIP11513                 0                   0                   0   
HIP08200                 0                   0                   0   
HIP00769                 0                   0                   0   
HIP09122                 0                   0                   0   
HIP12091         

In [3]:
public_result_with_tags_df=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/public_result_with_tags_df', "rb"))
public_result_with_tags_df.head()

,perc_public_df_0,perc_public_df_1,public10perc_df_0,public10perc_df_1,public50perc_df_0,public50perc_df_1,public95perc_df_0,public95perc_df_1,meanSharedSamples_df_0,meanSharedSamples_df_1,cdr3PriToPub_df_0,cdr3PriToPub_df_1,cdr3PriToPub95_df_0,cdr3PriToPub95_df_1,Age,Gender,_merge
Sample,,,,,,,,,,,,,,,,,
HIP09365,54.980117,7.120045,12.120690,0.037573,1.746403,0.0,0.024597,0.0,26.243074,1.291941,1.093374,1.170739,1.265726,NaN,43.0,Female,both
HIP05533,54.014608,5.991516,11.947385,0.026511,1.766843,0.0,0.025434,0.0,25.909560,1.274920,1.105388,1.180347,1.282926,NaN,22.0,Female,both
HIP13893,52.052226,5.884371,10.844156,0.017156,1.509827,0.0,0.020835,0.0,23.578078,1.244124,1.105201,1.197626,1.286461,NaN,33.0,Female,both
HIP13427,53.886438,7.218632,10.402663,0.036581,1.177351,0.0,0.014440,0.0,22.125660,1.280332,1.099480,1.175638,1.283347,NaN,42.0,Male,both
HIP01798,52.861126,6.713781,11.729311,0.110424,1.889867,0.0,0.030085,0.0,25.947240,1.353578,1.107016,1.186070,1.286112,NaN,NaN,Male,both


In [4]:
length_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/length_res_df_573_samples', "rb"))
clonality_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/clonality_res_df_573_samples', "rb"))
perProd_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/perProd_res_df_573_samples', "rb"))
general_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/general_res_df_573_samples', "rb"))
diversity_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/diversity_res_df_573_samples', "rb"))






In [6]:
combined_res_with_tags_df=pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(general_res_df_573_samples,perProd_res_df_573_samples,how='outer', 
                                                     right_index=True,left_index=True), clonality_res_df_573_samples,how='outer',
                                                      right_index=True,left_index=True),length_res_df_573_samples, how='outer',
                                                      right_index=True,left_index=True),diversity_res_df_573_samples, how='outer',
                                                      right_index=True,left_index=True),public_result_with_tags_df,how='outer',
                                                      right_index=True,left_index=True)

In [7]:
combined_res_with_tags_df.head()

,unique_seq_n_df_0,unique_seq_n_df_1,unique_aa_n_df_0,unique_aa_n_df_1,norm_uniqe_nt_sequences_df_0,norm_uniqe_nt_sequences_df_1,norm_uniqe_aa_sequences_df_0,norm_uniqe_aa_sequences_df_1,max_nt_per_aa_df_0,max_nt_per_aa_df_1,...,public95perc_df_1,meanSharedSamples_df_0,meanSharedSamples_df_1,cdr3PriToPub_df_0,cdr3PriToPub_df_1,cdr3PriToPub95_df_0,cdr3PriToPub95_df_1,Age,Gender,_merge
Sample,,,,,,,,,,,,,,,,,,,,,
HIP00110,104850.0,26090.0,96992.0,1884.0,44010.8,19067.0,4115.7,1152.9,31.0,7.0,...,0.0,28.901652,1.419321,1.103519,1.189689,1.270805,NaN,55.0,Male,both
HIP00169,96869.0,19050.0,92688.0,1545.0,50087.9,17846.4,4377.9,1332.3,12.0,2.0,...,0.0,27.809619,1.201294,1.107394,1.206712,1.280204,NaN,41.0,Male,both
HIP00594,161024.0,37806.0,150179.0,3002.0,48188.9,26489.6,3499.6,1174.5,27.0,7.0,...,0.0,28.911925,1.360093,1.102562,1.188030,1.270186,NaN,21.0,Male,both
HIP00602,197856.0,51103.0,180093.0,4216.0,52734.3,28421.6,4094.2,2085.0,36.0,6.0,...,0.0,26.791208,1.243121,1.103480,1.186119,1.287587,NaN,45.0,Female,both
HIP00614,105597.0,22677.0,96052.0,2070.0,46725.4,19452.8,4054.3,1431.1,147.0,17.0,...,0.0,28.050389,1.319324,1.105066,1.194228,1.278647,NaN,27.0,Male,both


In [8]:
combined_res_with_tags_df.describe()

,unique_seq_n_df_0,unique_seq_n_df_1,unique_aa_n_df_0,unique_aa_n_df_1,norm_uniqe_nt_sequences_df_0,norm_uniqe_nt_sequences_df_1,norm_uniqe_aa_sequences_df_0,norm_uniqe_aa_sequences_df_1,max_nt_per_aa_df_0,max_nt_per_aa_df_1,...,public50perc_df_1,public95perc_df_0,public95perc_df_1,meanSharedSamples_df_0,meanSharedSamples_df_1,cdr3PriToPub_df_0,cdr3PriToPub_df_1,cdr3PriToPub95_df_0,cdr3PriToPub95_df_1,Age
count,573.000000,573.000000,573.000000,573.000000,573.000000,573.000000,573.000000,573.000000,573.000000,573.000000,...,573.0,573.000000,573.0,573.000000,573.000000,573.000000,573.000000,573.000000,0.0,478.000000
mean,208440.048866,46224.511344,190153.650960,3819.968586,58042.258290,30092.395113,4254.177138,2009.649913,34.851658,5.432810,...,0.0,0.037685,0.0,27.939816,1.352501,1.098237,1.173858,1.268857,NaN,39.629707
std,80000.746462,19186.358571,71224.376475,1627.118109,11165.503936,9084.370591,460.388052,554.076114,44.401781,4.514286,...,0.0,0.018887,0.0,4.145224,0.469001,0.009979,0.026533,0.014638,NaN,14.200235
min,35144.000000,9287.000000,32555.000000,761.000000,24961.500000,8647.500000,2383.600000,531.000000,8.000000,2.000000,...,0.0,0.014440,0.0,20.848741,1.144513,1.040958,0.987952,1.190459,NaN,1.000000
25%,152274.000000,32069.000000,142200.000000,2667.000000,50270.500000,23766.300000,4009.400000,1655.300000,21.000000,3.000000,...,0.0,0.024956,0.0,25.496782,1.244866,1.095757,1.166037,1.264703,NaN,32.000000
50%,207174.000000,45158.000000,188920.000000,3700.000000,59331.600000,30364.700000,4363.500000,2031.700000,27.000000,4.000000,...,0.0,0.031913,0.0,27.238741,1.275382,1.099796,1.176864,1.270668,NaN,41.000000
75%,265181.000000,57121.000000,240657.000000,4746.000000,66334.100000,36064.600000,4587.200000,2390.100000,36.000000,6.000000,...,0.0,0.043516,0.0,29.233246,1.314836,1.103098,1.186908,1.276860,NaN,49.000000
max,471962.000000,114990.000000,415509.000000,10252.000000,83325.900000,56444.400000,4925.200000,3416.300000,593.000000,40.000000,...,0.0,0.181559,0.0,57.729121,5.703971,1.114193,1.231082,1.306098,NaN,74.000000


In [12]:
import time
cdate=str(time.strftime("%d%m%Y"))

In [13]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/combined_res_with_tags_df_%s' %cdate, 'wb') as f:
        pickle.dump(combined_res_with_tags_df,f)
f.close()